SIMULACIÓN DE UNA ESTACION DE SERVICIO
    
- SE TRATA DE UNA ESTACIÓN DE SERVICIO QUE TIENE UN DISPENSER DE COMBUSTIBLE, CON UNA CAPACIDAD DE 200 LITROS DE COMBUSTIBLE. SE TOMO ESTE VALOR PARA VER COMO FUNCIONA EL ALGORITMO, EN LA REALIDAD EL TANQUE DE UN SURTIDOR ES MUCHO MÁS GRANDE.
- LOS AUTOS LLEGAN CADA 150 SEGUNDOS.
- LA CANTIDAD QUE CARGAN LOS AUTOS ES ALEATORIA, DEPENDE DE CUANTOS LITROS TENGAN EN SU TANQUE ACTUALMENTE, SE TOMÓ COMO 50 LITROS LA CAPACIDAD DEL TANQUE LLENO DE TODOS LOS AUTOS.
- CUANDO EL TANQUE DE DISPENSER ESTE AL 15% DE SU CAPACIDAD O MENOS, SE LLAMARÁ AL CARRO CISTERNA PARA REABASTECER EL COMBUSTIBLE.

Importar librerias necesarias

In [59]:
import itertools
import random
import simpy

Definir variables

In [75]:
NUM_ALEATORIO = 42
CAP_TANQUE_ESTACION = 200    # Capacidad del tanque de la estación
LLAMAR_REABASTECER = 15      # Capacidad del tanque a la que se llamará al carro cisterna para reabastecimiento (en %)
TAM_TANQUE = 50              # Capacidad del tanque de combustible (en litros)
NIVEL_TANQUE = [5, 25]       # Nivel min/max del tanque de combustible (en litros)
VEL_LLENADO = 2              # Velocidad de llenado en litros/segundo
T_LLEGA_CISTERNA = 300       # Segundos que tarda el carro cisterna en llegar
T_LLEGAN_AUTOS = 150         # Crear un auto que llega a la estación cada 150 segundos
T_SIMULACION = 10000         # Tiempo de simulacion en segundos

Simulacion de la llegada de un auto a la estación de servicio, si no hay combustible en la 
estación, el auto esperará al carro cisterna a reabastecer el combustible

In [76]:
def auto(nombre, env, estacion_gas, dispenser_combustible):

    nivel_tanque_combustible = random.randint(*NIVEL_TANQUE)
    print('%s llega al surtidor a los %.1f segundos' % (nombre, env.now))
    with estacion_gas.request() as req:
        start = env.now
        yield req

        # Calculo de los litros requeridos del auto
        litros_requeridos = TAM_TANQUE - nivel_tanque_combustible
        yield dispenser_combustible.get(litros_requeridos)

        # Calculo del tiempo que tomará el llenado
        yield env.timeout(litros_requeridos / VEL_LLENADO)

        print('%s termina llenado de combustible en %.1f segundos.' % (nombre, env.now - start))

Periodicamente se revisará el nivel del dispenser de combustible y se llamará al carro cisterna 
si el nivel de combustible está al 15% o menos de la capacidad total del tanque

In [77]:
def control_estacion(env, dispenser_combustible):
    while True:
        if dispenser_combustible.level / dispenser_combustible.capacity * 100 < LLAMAR_REABASTECER:
            # Si se llega a este punto, es por que se debe llamar a la cisterna para reabastecimiento
            print('La estación llama a la cisterna a los %d segundos' % env.now)
            # Se esperará 300 segundos a que llegue el carro cisterna y a que llene el tanque de la estación
            yield env.process(cisterna(env, dispenser_combustible))

        # Se revisará el nivel del tanque cada 10 segundos
        yield env.timeout(20)  

    

La cisterna llega a la estación de servicio luego de un determinado tiempo y llena el tanque

In [78]:
def cisterna(env, dispenser_combustible):
    yield env.timeout(T_LLEGA_CISTERNA)
    print('El carro cisterna llega a los %d segundos' % env.now)
    ammount = dispenser_combustible.capacity - dispenser_combustible.level
    print('El carro cisterna carga %.1f litros al tanque.' % ammount)
    yield dispenser_combustible.put(ammount)




Generación de un auto que llega a estación de servicio:

In [79]:
def generar_auto(env, estacion_gas, dispenser_combustible):
    for i in itertools.count():
        yield env.timeout(T_LLEGAN_AUTOS)
        env.process(auto('Auto %d' % i, env, estacion_gas, dispenser_combustible))


Configuración y comienzo de la simulación

In [80]:
print('Simulación de Surtidor de combustible')
random.seed(NUM_ALEATORIO)
env = simpy.Environment()
estacion_gas = simpy.Resource(env, 2)
dispenser_combustible = simpy.Container(env, CAP_TANQUE_ESTACION, init=CAP_TANQUE_ESTACION)
env.process(control_estacion(env, dispenser_combustible))
env.process(generar_auto(env, estacion_gas, dispenser_combustible))

# Tiempo que durará la simulación
env.run(until=T_SIMULACION)

Simulación de Surtidor de combustible
Auto 0 llega al surtidor a los 150.0 segundos
Auto 0 termina llenado de combustible en 12.5 segundos.
Auto 1 llega al surtidor a los 300.0 segundos
Auto 1 termina llenado de combustible en 21.0 segundos.
Auto 2 llega al surtidor a los 450.0 segundos
Auto 2 termina llenado de combustible en 22.5 segundos.
Auto 3 llega al surtidor a los 600.0 segundos
Auto 3 termina llenado de combustible en 18.5 segundos.
Auto 4 llega al surtidor a los 750.0 segundos
La estación llama a la cisterna a los 760 segundos
Auto 4 termina llenado de combustible en 19.0 segundos.
Auto 5 llega al surtidor a los 900.0 segundos
Auto 6 llega al surtidor a los 1050.0 segundos
El carro cisterna llega a los 1060 segundos
El carro cisterna carga 187.0 litros al tanque.
Auto 5 termina llenado de combustible en 179.0 segundos.
Auto 6 termina llenado de combustible en 30.5 segundos.
Auto 7 llega al surtidor a los 1200.0 segundos
Auto 7 termina llenado de combustible en 21.0 segundos.
